In [8]:
from pyspark.sql.functions import from_json, col
import json
import pandas as pd
from pyspark.sql.functions import explode, split
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType, ArrayType, FloatType
import warnings
from pyspark.sql.functions import size
from pyspark.sql.functions import regexp_replace

### Pull in live game update data - DO NOT RE-RUN THIS SECTION

In [15]:
df = spark.read.parquet('/tmp/games')
df.registerTempTable('games')
query = """
create external table my_games
  stored as parquet
  location '/tmp/games_out'
  as
  select * from games
"""
spark.sql(query)

DataFrame[]

In [16]:
games2 = spark.read.parquet('/tmp/games_out')

In [18]:
extracted_games2 = games2.rdd.map(lambda x: json.loads(x.value)).toDF(schema=final_schema)

In [19]:
pd_games_stream = extracted_games2.toPandas()
pd_games_stream

,GameKey,Week,AwayTeam,AwayScore,HomeTeam,HomeScore,PointSpread,OverUnder,AwayTeamMoneyLine,HomeTeamMoneyLine
0,202111304,13,NE,None,BUF,None,-0.8,11.3,36,-43
1,202111304,13,NE,None,BUF,None,-0.8,11.3,36,-43


### Pull in historical batch data